<a href="https://colab.research.google.com/github/rosselladedo/DOG_BREEDS_SEARCH_ENGINE/blob/main/FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import numpy as np
import time
import itertools
import os, shutil
import tensorflow as tf
import matplotlib.pyplot as plt
import traceback
import sys
import sklearn
import pathlib
import random as rand

from IPython.display import display
from ipywidgets import HBox, Image, HTML

from sklearn import metrics
from sklearn.preprocessing import normalize

from random import random
from ipywidgets import Image
from numpy.linalg import norm
from IPython.display import display
from keras.models import Model
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tqdm.auto import tqdm

# tensorflow version 2.4.0
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from keras.applications.densenet  import preprocess_input

In [ ]:
DOGS_BREEDS = 120
IMAGE_SIZE = (224,224)
IMAGE_HEIGHT=224
IMAGE_WIDTH=224
BATCH_SIZE = 32
NUM_CLASSES= 120

In [ ]:
#data directories
BASE_DIR = "/content/gdrive/Shareddrives/MIRCV"
DATSETS= "/content/gdrive/Shareddrives/Dataset"

ALL_IMAGES_DIR = "/content/gdrive/Shareddrives/MIRCV/All_Images"
TEST_PATH = os.path.join(BASE_DIR, "TestSet")
DOGS_IMAGES_DIRECTORIES_PATH = "/content/gdrive/MyDrive/MIRCV/images/Images"

In [ ]:
#check hardware acceleration
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
!pip install split-folders tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
###### ESEGUIRE SOLO UNA VOLTA ############

! pip install -q kaggle

from google.colab import files
_ = files.upload()

! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [ ]:
#ESEGUIRE SOLO UNA VOLTA########
zip_path = '/content/gdrive/Shareddrives/Dataset/stanford-dogs-dataset.zip'
!cp "{zip_path}" .
!unzip stanford-dogs-dataset.zip -d '/content/gdrive/Shareddrives/Dataset' 
!rm stanford-dogs-dataset.zip

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DOGS_IMAGES_DIRECTORIES_PATH,
    labels='inferred', #the label of the dataset is obtained by the name of the directory
    label_mode="int",
    validation_split = 0.2,
    subset='training',
    seed=123,
    shuffle=True,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DOGS_IMAGES_DIRECTORIES_PATH,
    labels='inferred', #the label of the dataset is obtained by the name of the directory
    label_mode="int",
    validation_split = 0.2,
    subset='validation',
    seed=123,
    shuffle=True,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
)

Found 20580 files belonging to 120 classes.
Using 16464 files for training.
Found 20580 files belonging to 120 classes.
Using 4116 files for validation.


'\ntest_ds = tf.keras.preprocessing.image_dataset_from_directory(\n    SETS_DIR + \'/test\',\n    labels=\'inferred\', #the label of the dataset is obtained by the name of the directory\n    label_mode="int",\n    seed=123,\n    shuffle=False,\n    image_size=(224, 224),\n    batch_size=BATCH_SIZE,\n)\n'

In [ ]:
def preprocess(images, labels):
  image = tf.image.resize(images, IMAGE_SIZE)
  images = tf.keras.applications.densenet.preprocess_input(images)
  return images, labels
  
#preprocessing of the images in all the set
train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)
#test_ds = test_ds.map(preprocess)

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip('horizontal'),
        layers.RandomRotation(0.1, fill_mode="nearest"),
        layers.RandomZoom(0.2),
    ]
)

In [ ]:
pretrained_model = tf.keras.applications.DenseNet121(input_shape = (224,224,3),
                                                     weights="imagenet",
                                                     include_top=False) # do not include the pretrained layers implementing the imagenet classifier
pretrained_model.trainable = False 

29097984/29084464 [==============================] - 1s 0us/step


# **EXPERIMENT 1**

In [ ]:
############################################
#### FATTO #####
############################################

inputs = keras.Input(shape=(224,224,3))
x = data_augmentation (inputs)
x = pretrained_model(x)

x = layers.GlobalAveragePooling2D() (x)

x = layers.Dense(NUM_CLASSES, activation='softmax', name='dense_layer_120')(x)

model = keras.Model(inputs=inputs, outputs=x)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d_1   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_layer_120 (Dense)     (None, 120)               123000    
                                                                 
Total params: 7,160,504
Trainable params: 123,000
Non-trainable params: 7,037,504
___________________________________________

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = "/content/gdrive/Shareddrives/MIRCV/Models/model_1layer120_LR0001.h5" #il nome del modello è nel formato model_numerodese layer + sequenza dense layer_flatten oppure no_learning rate

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks=callbacks_list)

Epoch 1/25
258/258 [==============================] - ETA: 0s - loss: 1.5004 - accuracy: 0.6267
Epoch 1: val_loss improved from -inf to 0.82755, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_1layer120_LR0001.h5
258/258 [==============================] - 2235s 9s/step - loss: 1.5004 - accuracy: 0.6267 - val_loss: 0.8276 - val_accuracy: 0.7668
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 0.9055 - accuracy: 0.7481
Epoch 2: val_loss did not improve from 0.82755
258/258 [==============================] - 159s 611ms/step - loss: 0.9055 - accuracy: 0.7481 - val_loss: 0.6794 - val_accuracy: 0.7930
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 0.7568 - accuracy: 0.7801
Epoch 3: val_loss did not improve from 0.82755
258/258 [==============================] - 160s 613ms/step - loss: 0.7568 - accuracy: 0.7801 - val_loss: 0.6159 - val_accuracy: 0.8124
Epoch 4/25
258/258 [==============================] - ETA: 0s - loss: 0.6760 - acc

In [ ]:
'''
Epoch 1/25
258/258 [==============================] - ETA: 0s - loss: 1.5004 - accuracy: 0.6267
Epoch 1: val_loss improved from -inf to 0.82755, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_1layer120_LR0001.h5
258/258 [==============================] - 2235s 9s/step - loss: 1.5004 - accuracy: 0.6267 - val_loss: 0.8276 - val_accuracy: 0.7668
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 0.9055 - accuracy: 0.7481
Epoch 2: val_loss did not improve from 0.82755
258/258 [==============================] - 159s 611ms/step - loss: 0.9055 - accuracy: 0.7481 - val_loss: 0.6794 - val_accuracy: 0.7930
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 0.7568 - accuracy: 0.7801
Epoch 3: val_loss did not improve from 0.82755
258/258 [==============================] - 160s 613ms/step - loss: 0.7568 - accuracy: 0.7801 - val_loss: 0.6159 - val_accuracy: 0.8124
Epoch 4/25
258/258 [==============================] - ETA: 0s - loss: 0.6760 - accuracy: 0.7973
Epoch 4: val_loss did not improve from 0.82755
258/258 [==============================] - 160s 613ms/step - loss: 0.6760 - accuracy: 0.7973 - val_loss: 0.5895 - val_accuracy: 0.8139
Epoch 5/25
258/258 [==============================] - ETA: 0s - loss: 0.6266 - accuracy: 0.8166
Epoch 5: val_loss did not improve from 0.82755
258/258 [==============================] - 160s 613ms/step - loss: 0.6266 - accuracy: 0.8166 - val_loss: 0.5772 - val_accuracy: 0.8163
Epoch 6/25
258/258 [==============================] - ETA: 0s - loss: 0.5818 - accuracy: 0.8238
Epoch 6: val_loss did not improve from 0.82755
258/258 [==============================] - 160s 613ms/step - loss: 0.5818 - accuracy: 0.8238 - val_loss: 0.5676 - val_accuracy: 0.8219
Epoch 7/25
258/258 [==============================] - ETA: 0s - loss: 0.5576 - accuracy: 0.8308
Epoch 7: val_loss did not improve from 0.82755
258/258 [==============================] - 160s 613ms/step - loss: 0.5576 - accuracy: 0.8308 - val_loss: 0.5630 - val_accuracy: 0.8168
Epoch 8/25
258/258 [==============================] - ETA: 0s - loss: 0.5338 - accuracy: 0.8363
Epoch 8: val_loss did not improve from 0.82755
258/258 [==============================] - 160s 613ms/step - loss: 0.5338 - accuracy: 0.8363 - val_loss: 0.5571 - val_accuracy: 0.8219
Epoch 9/25
258/258 [==============================] - ETA: 0s - loss: 0.4991 - accuracy: 0.8454
Epoch 9: val_loss did not improve from 0.82755
258/258 [==============================] - 160s 613ms/step - loss: 0.4991 - accuracy: 0.8454 - val_loss: 0.5632 - val_accuracy: 0.8161
Epoch 10/25
258/258 [==============================] - ETA: 0s - loss: 0.4775 - accuracy: 0.8533
Epoch 10: val_loss did not improve from 0.82755
258/258 [==============================] - 160s 613ms/step - loss: 0.4775 - accuracy: 0.8533 - val_loss: 0.5651 - val_accuracy: 0.8178
'''

# **EXPERIMENT 2**

In [ ]:
############################################
#### FATTO #####
############################################

inputs = keras.Input(shape=(224,224,3))
x = data_augmentation (inputs)
x = pretrained_model(x)

x = layers.GlobalAveragePooling2D() (x)

x = layers.Flatten(name='flatten')(x)

x = layers.Dense(NUM_CLASSES, activation='softmax', name='dense_layer_120')(x)

model = keras.Model(inputs=inputs, outputs=x)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense_layer_120 (Dense)     (None, 120)               123000    
                                                             

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = "/content/gdrive/Shareddrives/MIRCV/Models/model_1layer120_Flatten_LR0001.h5" #il nome del modello è nel formato model_numerodese layer + sequenza dense layer_flatten oppure no_learning rate

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks=callbacks_list)

Epoch 1/25
258/258 [==============================] - ETA: 0s - loss: 1.6591 - accuracy: 0.5783
Epoch 1: val_loss improved from -inf to 0.82203, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_1layer120_Flatten_LR0001.h5
258/258 [==============================] - 2487s 9s/step - loss: 1.6591 - accuracy: 0.5783 - val_loss: 0.8220 - val_accuracy: 0.7493
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 0.9686 - accuracy: 0.7240
Epoch 2: val_loss did not improve from 0.82203
258/258 [==============================] - 74s 282ms/step - loss: 0.9686 - accuracy: 0.7240 - val_loss: 0.7723 - val_accuracy: 0.7787
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 0.8419 - accuracy: 0.7589
Epoch 3: val_loss did not improve from 0.82203
258/258 [==============================] - 74s 281ms/step - loss: 0.8419 - accuracy: 0.7589 - val_loss: 0.8128 - val_accuracy: 0.7796
Epoch 4/25
258/258 [==============================] - ETA: 0s - loss: 0.8253

In [ ]:
'''
Epoch 1/25
258/258 [==============================] - ETA: 0s - loss: 1.6591 - accuracy: 0.5783
Epoch 1: val_loss improved from -inf to 0.82203, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_1layer120_Flatten_LR0001.h5
258/258 [==============================] - 2487s 9s/step - loss: 1.6591 - accuracy: 0.5783 - val_loss: 0.8220 - val_accuracy: 0.7493
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 0.9686 - accuracy: 0.7240
Epoch 2: val_loss did not improve from 0.82203
258/258 [==============================] - 74s 282ms/step - loss: 0.9686 - accuracy: 0.7240 - val_loss: 0.7723 - val_accuracy: 0.7787
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 0.8419 - accuracy: 0.7589
Epoch 3: val_loss did not improve from 0.82203
258/258 [==============================] - 74s 281ms/step - loss: 0.8419 - accuracy: 0.7589 - val_loss: 0.8128 - val_accuracy: 0.7796
Epoch 4/25
258/258 [==============================] - ETA: 0s - loss: 0.8253 - accuracy: 0.7734
Epoch 4: val_loss did not improve from 0.82203
258/258 [==============================] - 73s 280ms/step - loss: 0.8253 - accuracy: 0.7734 - val_loss: 0.8102 - val_accuracy: 0.7869
'''

# **EXPERIMENT 3**

In [ ]:
############################################
#### FATTO #####
############################################

inputs = keras.Input(shape=(224,224,3))
x = data_augmentation (inputs)
x = pretrained_model(x)

x = layers.GlobalAveragePooling2D() (x)

x = layers.Dense(256, activation='relu', name='dense_layer_256')(x)

x = layers.Dense(NUM_CLASSES, activation='softmax', name='dense_layer_120')(x)

model = keras.Model(inputs=inputs, outputs=x)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense_layer_256 (Dense)     (None, 256)               262400    
                                                                 
 dense_layer_120 (Dense)     (None, 120)               30840     
                                                             

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = "/content/gdrive/Shareddrives/MIRCV/Models/model_2layer256120_LR0001.h5" #il nome del modello è nel formato model_numerodese layer + sequenza dense layer_flatten oppure no_learning rate

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks=callbacks_list)

Epoch 1/25
258/258 [==============================] - ETA: 0s - loss: 1.9328 - accuracy: 0.5196
Epoch 1: val_loss improved from -inf to 0.79904, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_2layer256120_LR0001.h5
258/258 [==============================] - 83s 296ms/step - loss: 1.9328 - accuracy: 0.5196 - val_loss: 0.7990 - val_accuracy: 0.7464
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 0.9040 - accuracy: 0.7289
Epoch 2: val_loss did not improve from 0.79904
258/258 [==============================] - 75s 285ms/step - loss: 0.9040 - accuracy: 0.7289 - val_loss: 0.7049 - val_accuracy: 0.7767
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 0.7990 - accuracy: 0.7533
Epoch 3: val_loss did not improve from 0.79904
258/258 [==============================] - 74s 284ms/step - loss: 0.7990 - accuracy: 0.7533 - val_loss: 0.6429 - val_accuracy: 0.7913
Epoch 4/25
258/258 [==============================] - ETA: 0s - loss: 0.7195 - a

In [ ]:
'''
Epoch 1/25
258/258 [==============================] - ETA: 0s - loss: 1.9328 - accuracy: 0.5196
Epoch 1: val_loss improved from -inf to 0.79904, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_2layer256120_LR0001.h5
258/258 [==============================] - 83s 296ms/step - loss: 1.9328 - accuracy: 0.5196 - val_loss: 0.7990 - val_accuracy: 0.7464
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 0.9040 - accuracy: 0.7289
Epoch 2: val_loss did not improve from 0.79904
258/258 [==============================] - 75s 285ms/step - loss: 0.9040 - accuracy: 0.7289 - val_loss: 0.7049 - val_accuracy: 0.7767
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 0.7990 - accuracy: 0.7533
Epoch 3: val_loss did not improve from 0.79904
258/258 [==============================] - 74s 284ms/step - loss: 0.7990 - accuracy: 0.7533 - val_loss: 0.6429 - val_accuracy: 0.7913
Epoch 4/25
258/258 [==============================] - ETA: 0s - loss: 0.7195 - accuracy: 0.7730
Epoch 4: val_loss did not improve from 0.79904
258/258 [==============================] - 74s 284ms/step - loss: 0.7195 - accuracy: 0.7730 - val_loss: 0.6471 - val_accuracy: 0.7947
Epoch 5/25
258/258 [==============================] - ETA: 0s - loss: 0.6712 - accuracy: 0.7878
Epoch 5: val_loss did not improve from 0.79904
258/258 [==============================] - 74s 284ms/step - loss: 0.6712 - accuracy: 0.7878 - val_loss: 0.6379 - val_accuracy: 0.7988
Epoch 6/25
258/258 [==============================] - ETA: 0s - loss: 0.6410 - accuracy: 0.7956
Epoch 6: val_loss did not improve from 0.79904
258/258 [==============================] - 74s 283ms/step - loss: 0.6410 - accuracy: 0.7956 - val_loss: 0.6418 - val_accuracy: 0.7969
Epoch 7/25
258/258 [==============================] - ETA: 0s - loss: 0.6061 - accuracy: 0.8092
Epoch 7: val_loss did not improve from 0.79904
258/258 [==============================] - 75s 284ms/step - loss: 0.6061 - accuracy: 0.8092 - val_loss: 0.6177 - val_accuracy: 0.8083
Epoch 8/25
258/258 [==============================] - ETA: 0s - loss: 0.5659 - accuracy: 0.8204
Epoch 8: val_loss did not improve from 0.79904
258/258 [==============================] - 80s 306ms/step - loss: 0.5659 - accuracy: 0.8204 - val_loss: 0.6294 - val_accuracy: 0.8039
Epoch 9/25
258/258 [==============================] - ETA: 0s - loss: 0.5328 - accuracy: 0.8254
Epoch 9: val_loss did not improve from 0.79904
258/258 [==============================] - 75s 284ms/step - loss: 0.5328 - accuracy: 0.8254 - val_loss: 0.6249 - val_accuracy: 0.8132
'''

# **EXPERIMENT 4**

In [ ]:
############################################
#### FATTO #####
############################################

inputs = keras.Input(shape=(224,224,3))
x = data_augmentation (inputs)
x = pretrained_model(x)

x = layers.GlobalAveragePooling2D() (x)

x = layers.Flatten(name='flatten')(x)

x = layers.Dense(256, activation='relu', name='dense_layer_256')(x)

x = layers.Dense(NUM_CLASSES, activation='softmax', name='dense_layer_120')(x)

model = keras.Model(inputs=inputs, outputs=x)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d_1   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense_layer_256 (Dense)     (None, 256)               262400    
                                                           

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = "/content/gdrive/Shareddrives/MIRCV/Models/model_2layer256120_Flatten_LR0001.h5" #il nome del modello è nel formato model_numerodese layer + sequenza dense layer_flatten oppure no_learning rate

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks=callbacks_list)

Epoch 1/25
258/258 [==============================] - ETA: 0s - loss: 1.9172 - accuracy: 0.5268
Epoch 1: val_loss improved from -inf to 0.82801, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_2layer256120_Flatten_LR0001.h5
258/258 [==============================] - 81s 288ms/step - loss: 1.9172 - accuracy: 0.5268 - val_loss: 0.8280 - val_accuracy: 0.7461
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 0.9147 - accuracy: 0.7249
Epoch 2: val_loss did not improve from 0.82801
258/258 [==============================] - 74s 282ms/step - loss: 0.9147 - accuracy: 0.7249 - val_loss: 0.6825 - val_accuracy: 0.7867
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 0.7900 - accuracy: 0.7569
Epoch 3: val_loss did not improve from 0.82801
258/258 [==============================] - 74s 280ms/step - loss: 0.7900 - accuracy: 0.7569 - val_loss: 0.6278 - val_accuracy: 0.8000
Epoch 4/25
258/258 [==============================] - ETA: 0s - loss: 0.

In [ ]:
'''
Epoch 1/25
258/258 [==============================] - ETA: 0s - loss: 1.9172 - accuracy: 0.5268
Epoch 1: val_loss improved from -inf to 0.82801, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_2layer256120_Flatten_LR0001.h5
258/258 [==============================] - 81s 288ms/step - loss: 1.9172 - accuracy: 0.5268 - val_loss: 0.8280 - val_accuracy: 0.7461
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 0.9147 - accuracy: 0.7249
Epoch 2: val_loss did not improve from 0.82801
258/258 [==============================] - 74s 282ms/step - loss: 0.9147 - accuracy: 0.7249 - val_loss: 0.6825 - val_accuracy: 0.7867
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 0.7900 - accuracy: 0.7569
Epoch 3: val_loss did not improve from 0.82801
258/258 [==============================] - 74s 280ms/step - loss: 0.7900 - accuracy: 0.7569 - val_loss: 0.6278 - val_accuracy: 0.8000
Epoch 4/25
258/258 [==============================] - ETA: 0s - loss: 0.7257 - accuracy: 0.7742
Epoch 4: val_loss did not improve from 0.82801
258/258 [==============================] - 73s 279ms/step - loss: 0.7257 - accuracy: 0.7742 - val_loss: 0.6071 - val_accuracy: 0.8107
Epoch 5/25
258/258 [==============================] - ETA: 0s - loss: 0.6682 - accuracy: 0.7920
Epoch 5: val_loss did not improve from 0.82801
258/258 [==============================] - 79s 301ms/step - loss: 0.6682 - accuracy: 0.7920 - val_loss: 0.6198 - val_accuracy: 0.8032
Epoch 6/25
258/258 [==============================] - ETA: 0s - loss: 0.6434 - accuracy: 0.7966
Epoch 6: val_loss did not improve from 0.82801
258/258 [==============================] - 74s 280ms/step - loss: 0.6434 - accuracy: 0.7966 - val_loss: 0.6161 - val_accuracy: 0.8088
'''

# **EXPERIMENT 5**

In [ ]:
############################################
#### FATTO #####
############################################

inputs = keras.Input(shape=(224,224,3))
x = data_augmentation (inputs)
x = pretrained_model(x)

x = layers.GlobalAveragePooling2D() (x)

x = layers.Dense(512, activation='relu', name='dense_layer_512')(x)

x = layers.Dense(NUM_CLASSES, activation='softmax', name='dense_layer_120')(x)

model = keras.Model(inputs=inputs, outputs=x)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d_2   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_layer_512 (Dense)     (None, 512)               524800    
                                                                 
 dense_layer_120 (Dense)     (None, 120)               61560     
                                                           

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = "/content/gdrive/Shareddrives/MIRCV/Models/model_2layer512120_LR0001.h5" #il nome del modello è nel formato model_numerodese layer + sequenza dense layer_flatten oppure no_learning rate

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks=callbacks_list)

Epoch 1/25
258/258 [==============================] - ETA: 0s - loss: 1.6551 - accuracy: 0.5748
Epoch 1: val_loss improved from -inf to 0.76382, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_2layer512120_LR0001.h5
258/258 [==============================] - 82s 292ms/step - loss: 1.6551 - accuracy: 0.5748 - val_loss: 0.7638 - val_accuracy: 0.7536
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 0.8690 - accuracy: 0.7384
Epoch 2: val_loss did not improve from 0.76382
258/258 [==============================] - 74s 282ms/step - loss: 0.8690 - accuracy: 0.7384 - val_loss: 0.6892 - val_accuracy: 0.7762
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 0.7526 - accuracy: 0.7657
Epoch 3: val_loss did not improve from 0.76382
258/258 [==============================] - 74s 283ms/step - loss: 0.7526 - accuracy: 0.7657 - val_loss: 0.6278 - val_accuracy: 0.7976
Epoch 4/25
258/258 [==============================] - ETA: 0s - loss: 0.6897 - a

In [ ]:
'''
Epoch 1/25
258/258 [==============================] - ETA: 0s - loss: 1.6551 - accuracy: 0.5748
Epoch 1: val_loss improved from -inf to 0.76382, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_2layer512120_LR0001.h5
258/258 [==============================] - 82s 292ms/step - loss: 1.6551 - accuracy: 0.5748 - val_loss: 0.7638 - val_accuracy: 0.7536
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 0.8690 - accuracy: 0.7384
Epoch 2: val_loss did not improve from 0.76382
258/258 [==============================] - 74s 282ms/step - loss: 0.8690 - accuracy: 0.7384 - val_loss: 0.6892 - val_accuracy: 0.7762
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 0.7526 - accuracy: 0.7657
Epoch 3: val_loss did not improve from 0.76382
258/258 [==============================] - 74s 283ms/step - loss: 0.7526 - accuracy: 0.7657 - val_loss: 0.6278 - val_accuracy: 0.7976
Epoch 4/25
258/258 [==============================] - ETA: 0s - loss: 0.6897 - accuracy: 0.7835
Epoch 4: val_loss did not improve from 0.76382
258/258 [==============================] - 74s 283ms/step - loss: 0.6897 - accuracy: 0.7835 - val_loss: 0.6240 - val_accuracy: 0.7954
Epoch 5/25
258/258 [==============================] - ETA: 0s - loss: 0.6464 - accuracy: 0.7935
Epoch 5: val_loss did not improve from 0.76382
258/258 [==============================] - 74s 282ms/step - loss: 0.6464 - accuracy: 0.7935 - val_loss: 0.6032 - val_accuracy: 0.8044
Epoch 6/25
258/258 [==============================] - ETA: 0s - loss: 0.5976 - accuracy: 0.8089
Epoch 6: val_loss did not improve from 0.76382
258/258 [==============================] - 74s 282ms/step - loss: 0.5976 - accuracy: 0.8089 - val_loss: 0.6196 - val_accuracy: 0.7976
Epoch 7/25
258/258 [==============================] - ETA: 0s - loss: 0.5653 - accuracy: 0.8140
Epoch 7: val_loss did not improve from 0.76382
258/258 [==============================] - 80s 305ms/step - loss: 0.5653 - accuracy: 0.8140 - val_loss: 0.6048 - val_accuracy: 0.8117
'''

# **EXPERTIMENT 6**

In [ ]:
############################################
#### FATTO #####
############################################

inputs = keras.Input(shape=(224,224,3))
x = data_augmentation (inputs)
x = pretrained_model(x)

x = layers.GlobalAveragePooling2D() (x)

x = layers.Flatten(name='flatten')(x)

x = layers.Dense(512, activation='relu', name='dense_layer_512')(x)

x = layers.Dense(NUM_CLASSES, activation='softmax', name='dense_layer_120')(x)

model = keras.Model(inputs=inputs, outputs=x)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d_3   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense_layer_512 (Dense)     (None, 512)               524800    
                                                           

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = "/content/gdrive/Shareddrives/MIRCV/Models/model_2layer512120_Flatten_LR0001.h5" #il nome del modello è nel formato model_numerodese layer + sequenza dense layer_flatten oppure no_learning rate

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks=callbacks_list)

Epoch 1/25
258/258 [==============================] - ETA: 0s - loss: 1.6517 - accuracy: 0.5719
Epoch 1: val_loss improved from -inf to 0.73541, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_2layer512120_Flatten_LR0001.h5
258/258 [==============================] - 88s 317ms/step - loss: 1.6517 - accuracy: 0.5719 - val_loss: 0.7354 - val_accuracy: 0.7634
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 0.8719 - accuracy: 0.7354
Epoch 2: val_loss did not improve from 0.73541
258/258 [==============================] - 74s 281ms/step - loss: 0.8719 - accuracy: 0.7354 - val_loss: 0.6924 - val_accuracy: 0.7794
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 0.7566 - accuracy: 0.7618
Epoch 3: val_loss did not improve from 0.73541
258/258 [==============================] - 73s 278ms/step - loss: 0.7566 - accuracy: 0.7618 - val_loss: 0.6460 - val_accuracy: 0.7964
Epoch 4/25
258/258 [==============================] - ETA: 0s - loss: 0.

In [ ]:
'''
Epoch 1/25
258/258 [==============================] - ETA: 0s - loss: 1.6517 - accuracy: 0.5719
Epoch 1: val_loss improved from -inf to 0.73541, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_2layer512120_Flatten_LR0001.h5
258/258 [==============================] - 88s 317ms/step - loss: 1.6517 - accuracy: 0.5719 - val_loss: 0.7354 - val_accuracy: 0.7634
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 0.8719 - accuracy: 0.7354
Epoch 2: val_loss did not improve from 0.73541
258/258 [==============================] - 74s 281ms/step - loss: 0.8719 - accuracy: 0.7354 - val_loss: 0.6924 - val_accuracy: 0.7794
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 0.7566 - accuracy: 0.7618
Epoch 3: val_loss did not improve from 0.73541
258/258 [==============================] - 73s 278ms/step - loss: 0.7566 - accuracy: 0.7618 - val_loss: 0.6460 - val_accuracy: 0.7964
Epoch 4/25
258/258 [==============================] - ETA: 0s - loss: 0.6793 - accuracy: 0.7872
Epoch 4: val_loss did not improve from 0.73541
258/258 [==============================] - 73s 277ms/step - loss: 0.6793 - accuracy: 0.7872 - val_loss: 0.6416 - val_accuracy: 0.7964
Epoch 5/25
258/258 [==============================] - ETA: 0s - loss: 0.6366 - accuracy: 0.7969
Epoch 5: val_loss did not improve from 0.73541
258/258 [==============================] - 74s 281ms/step - loss: 0.6366 - accuracy: 0.7969 - val_loss: 0.6483 - val_accuracy: 0.7952
Epoch 6/25
258/258 [==============================] - ETA: 0s - loss: 0.5982 - accuracy: 0.8087
Epoch 6: val_loss did not improve from 0.73541
258/258 [==============================] - 74s 282ms/step - loss: 0.5982 - accuracy: 0.8087 - val_loss: 0.6369 - val_accuracy: 0.7918
Epoch 7/25
258/258 [==============================] - ETA: 0s - loss: 0.5442 - accuracy: 0.8245
Epoch 7: val_loss did not improve from 0.73541
258/258 [==============================] - 74s 282ms/step - loss: 0.5442 - accuracy: 0.8245 - val_loss: 0.6514 - val_accuracy: 0.7981
Epoch 8/25
258/258 [==============================] - ETA: 0s - loss: 0.5348 - accuracy: 0.8265
Epoch 8: val_loss did not improve from 0.73541
258/258 [==============================] - 74s 283ms/step - loss: 0.5348 - accuracy: 0.8265 - val_loss: 0.6545 - val_accuracy: 0.7962
'''

# **EXPERIMENT 7**

In [ ]:
############################################
#### FATTO #####
############################################

inputs = keras.Input(shape=(224,224,3))
x = data_augmentation (inputs)
x = pretrained_model(x)

x = layers.GlobalAveragePooling2D() (x)

x = layers.Dense(512, activation='relu', name='dense_layer_512')(x)

x = layers.Dense(256, activation='relu', name='dense_layer_256')(x)

x = layers.Dense(NUM_CLASSES, activation='softmax', name='dense_layer_120')(x)

model = keras.Model(inputs=inputs, outputs=x)
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d_4   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_layer_512 (Dense)     (None, 512)               524800    
                                                                 
 dense_layer_256 (Dense)     (None, 256)               131328    
                                                           

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = "/content/gdrive/Shareddrives/MIRCV/Models/model_3layer512256120_LR0001.h5" #il nome del modello è nel formato model_numerodese layer + sequenza dense layer_flatten oppure no_learning rate

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks=callbacks_list)

Epoch 1/25
258/258 [==============================] - ETA: 0s - loss: 1.8010 - accuracy: 0.5311
Epoch 1: val_loss improved from -inf to 0.79111, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_3layer512256120_LR0001.h5
258/258 [==============================] - 82s 294ms/step - loss: 1.8010 - accuracy: 0.5311 - val_loss: 0.7911 - val_accuracy: 0.7493
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 0.9399 - accuracy: 0.7138
Epoch 2: val_loss did not improve from 0.79111
258/258 [==============================] - 76s 288ms/step - loss: 0.9399 - accuracy: 0.7138 - val_loss: 0.7404 - val_accuracy: 0.7607
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 0.8255 - accuracy: 0.7430
Epoch 3: val_loss did not improve from 0.79111
258/258 [==============================] - 74s 283ms/step - loss: 0.8255 - accuracy: 0.7430 - val_loss: 0.6887 - val_accuracy: 0.7799
Epoch 4/25
258/258 [==============================] - ETA: 0s - loss: 0.7543 

In [ ]:
'''
Epoch 1/25
258/258 [==============================] - ETA: 0s - loss: 1.8010 - accuracy: 0.5311
Epoch 1: val_loss improved from -inf to 0.79111, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_3layer512256120_LR0001.h5
258/258 [==============================] - 82s 294ms/step - loss: 1.8010 - accuracy: 0.5311 - val_loss: 0.7911 - val_accuracy: 0.7493
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 0.9399 - accuracy: 0.7138
Epoch 2: val_loss did not improve from 0.79111
258/258 [==============================] - 76s 288ms/step - loss: 0.9399 - accuracy: 0.7138 - val_loss: 0.7404 - val_accuracy: 0.7607
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 0.8255 - accuracy: 0.7430
Epoch 3: val_loss did not improve from 0.79111
258/258 [==============================] - 74s 283ms/step - loss: 0.8255 - accuracy: 0.7430 - val_loss: 0.6887 - val_accuracy: 0.7799
Epoch 4/25
258/258 [==============================] - ETA: 0s - loss: 0.7543 - accuracy: 0.7653
Epoch 4: val_loss did not improve from 0.79111
258/258 [==============================] - 80s 306ms/step - loss: 0.7543 - accuracy: 0.7653 - val_loss: 0.7222 - val_accuracy: 0.7755
Epoch 5/25
258/258 [==============================] - ETA: 0s - loss: 0.7047 - accuracy: 0.7775
Epoch 5: val_loss did not improve from 0.79111
258/258 [==============================] - 75s 286ms/step - loss: 0.7047 - accuracy: 0.7775 - val_loss: 0.6906 - val_accuracy: 0.7838
'''

# **EXPERIMENT 8**

In [ ]:
############################################
#### FATTO #####
############################################

inputs = keras.Input(shape=(224,224,3))
x = data_augmentation (inputs)
x = pretrained_model(x)

x = layers.GlobalAveragePooling2D() (x)

x = layers.Flatten(name='flatten')(x)

x = layers.Dense(512, activation='relu', name='dense_layer_512')(x)

x = layers.Dense(256, activation='relu', name='dense_layer_256')(x)

x = layers.Dense(NUM_CLASSES, activation='softmax', name='dense_layer_120')(x)

model = keras.Model(inputs=inputs, outputs=x)
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d_5   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense_layer_512 (Dense)     (None, 512)               524800    
                                                           

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = "/content/gdrive/Shareddrives/MIRCV/Models/model_3layer512256120_Flatten_LR0001.h5" #il nome del modello è nel formato model_numerodese layer + sequenza dense layer_flatten oppure no_learning rate

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks=callbacks_list)

Epoch 1/25
258/258 [==============================] - ETA: 0s - loss: 1.7741 - accuracy: 0.5363
Epoch 1: val_loss improved from -inf to 0.78455, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_3layer512256120_Flatten_LR0001.h5
258/258 [==============================] - 83s 297ms/step - loss: 1.7741 - accuracy: 0.5363 - val_loss: 0.7846 - val_accuracy: 0.7534
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 0.9354 - accuracy: 0.7129
Epoch 2: val_loss did not improve from 0.78455
258/258 [==============================] - 75s 287ms/step - loss: 0.9354 - accuracy: 0.7129 - val_loss: 0.7561 - val_accuracy: 0.7578
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 0.8300 - accuracy: 0.7418
Epoch 3: val_loss did not improve from 0.78455
258/258 [==============================] - 75s 286ms/step - loss: 0.8300 - accuracy: 0.7418 - val_loss: 0.6707 - val_accuracy: 0.7821
Epoch 4/25
258/258 [==============================] - ETA: 0s - loss:

In [ ]:
'''
Epoch 1/25
258/258 [==============================] - ETA: 0s - loss: 1.7741 - accuracy: 0.5363
Epoch 1: val_loss improved from -inf to 0.78455, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_3layer512256120_Flatten_LR0001.h5
258/258 [==============================] - 83s 297ms/step - loss: 1.7741 - accuracy: 0.5363 - val_loss: 0.7846 - val_accuracy: 0.7534
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 0.9354 - accuracy: 0.7129
Epoch 2: val_loss did not improve from 0.78455
258/258 [==============================] - 75s 287ms/step - loss: 0.9354 - accuracy: 0.7129 - val_loss: 0.7561 - val_accuracy: 0.7578
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 0.8300 - accuracy: 0.7418
Epoch 3: val_loss did not improve from 0.78455
258/258 [==============================] - 75s 286ms/step - loss: 0.8300 - accuracy: 0.7418 - val_loss: 0.6707 - val_accuracy: 0.7821
Epoch 4/25
258/258 [==============================] - ETA: 0s - loss: 0.7638 - accuracy: 0.7622
Epoch 4: val_loss did not improve from 0.78455
258/258 [==============================] - 80s 304ms/step - loss: 0.7638 - accuracy: 0.7622 - val_loss: 0.7070 - val_accuracy: 0.7787
Epoch 5/25
258/258 [==============================] - ETA: 0s - loss: 0.6928 - accuracy: 0.7837
Epoch 5: val_loss did not improve from 0.78455
258/258 [==============================] - 75s 286ms/step - loss: 0.6928 - accuracy: 0.7837 - val_loss: 0.6962 - val_accuracy: 0.7743
'''

# **EXPERIMENT 9**

In [ ]:
############################################
#### FATTO #####
############################################

inputs = keras.Input(shape=(224,224,3))
x = data_augmentation (inputs)
x = pretrained_model(x)

x = layers.GlobalAveragePooling2D() (x)

x = layers.Dense(1024, activation='relu', name='dense_layer_1024')(x)

x = layers.Dense(512, activation='relu', name='dense_layer_512')(x)

x = layers.Dense(256, activation='relu', name='dense_layer_256')(x)

x = layers.Dense(NUM_CLASSES, activation='softmax', name='dense_layer_120')(x)

model = keras.Model(inputs=inputs, outputs=x)
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d_6   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_layer_1024 (Dense)    (None, 1024)              1049600   
                                                                 
 dense_layer_512 (Dense)     (None, 512)               524800    
                                                           

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = "/content/gdrive/Shareddrives/MIRCV/Models/model_4layer1024512256120_LR0001.h5" #il nome del modello è nel formato model_numerodese layer + sequenza dense layer_flatten oppure no_learning rate

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks=callbacks_list)

Epoch 1/25
258/258 [==============================] - ETA: 0s - loss: 1.8177 - accuracy: 0.5193
Epoch 1: val_loss improved from -inf to 0.92300, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_4layer1024512256120_LR0001.h5
258/258 [==============================] - 83s 297ms/step - loss: 1.8177 - accuracy: 0.5193 - val_loss: 0.9230 - val_accuracy: 0.7024
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 1.0212 - accuracy: 0.6875
Epoch 2: val_loss did not improve from 0.92300
258/258 [==============================] - 81s 310ms/step - loss: 1.0212 - accuracy: 0.6875 - val_loss: 0.8405 - val_accuracy: 0.7328
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 0.9072 - accuracy: 0.7194
Epoch 3: val_loss did not improve from 0.92300
258/258 [==============================] - 75s 288ms/step - loss: 0.9072 - accuracy: 0.7194 - val_loss: 0.7762 - val_accuracy: 0.7568
Epoch 4/25
258/258 [==============================] - ETA: 0s - loss: 0.8

In [ ]:
'''
Epoch 1/25
258/258 [==============================] - ETA: 0s - loss: 1.8177 - accuracy: 0.5193
Epoch 1: val_loss improved from -inf to 0.92300, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_4layer1024512256120_LR0001.h5
258/258 [==============================] - 83s 297ms/step - loss: 1.8177 - accuracy: 0.5193 - val_loss: 0.9230 - val_accuracy: 0.7024
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 1.0212 - accuracy: 0.6875
Epoch 2: val_loss did not improve from 0.92300
258/258 [==============================] - 81s 310ms/step - loss: 1.0212 - accuracy: 0.6875 - val_loss: 0.8405 - val_accuracy: 0.7328
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 0.9072 - accuracy: 0.7194
Epoch 3: val_loss did not improve from 0.92300
258/258 [==============================] - 75s 288ms/step - loss: 0.9072 - accuracy: 0.7194 - val_loss: 0.7762 - val_accuracy: 0.7568
Epoch 4/25
258/258 [==============================] - ETA: 0s - loss: 0.8033 - accuracy: 0.7501
Epoch 4: val_loss did not improve from 0.92300
258/258 [==============================] - 76s 288ms/step - loss: 0.8033 - accuracy: 0.7501 - val_loss: 0.7357 - val_accuracy: 0.7690
Epoch 5/25
258/258 [==============================] - ETA: 0s - loss: 0.7576 - accuracy: 0.7602
Epoch 5: val_loss did not improve from 0.92300
258/258 [==============================] - 75s 285ms/step - loss: 0.7576 - accuracy: 0.7602 - val_loss: 0.6981 - val_accuracy: 0.7804
Epoch 6/25
258/258 [==============================] - ETA: 0s - loss: 0.7053 - accuracy: 0.7747
Epoch 6: val_loss did not improve from 0.92300
258/258 [==============================] - 74s 284ms/step - loss: 0.7053 - accuracy: 0.7747 - val_loss: 0.7707 - val_accuracy: 0.7636
Epoch 7/25
258/258 [==============================] - ETA: 0s - loss: 0.6690 - accuracy: 0.7831
Epoch 7: val_loss did not improve from 0.92300
258/258 [==============================] - 75s 284ms/step - loss: 0.6690 - accuracy: 0.7831 - val_loss: 0.7544 - val_accuracy: 0.7755
'''

# **EXPERIMENT 10**

In [ ]:
############################################
#### FATTO #####
############################################

inputs = keras.Input(shape=(224,224,3))
x = data_augmentation (inputs)
x = pretrained_model(x)

x = layers.GlobalAveragePooling2D() (x)

x = layers.Flatten(name='flatten')(x)

x = layers.Dense(1024, activation='relu', name='dense_layer_1024')(x)

x = layers.Dense(512, activation='relu', name='dense_layer_512')(x)

x = layers.Dense(256, activation='relu', name='dense_layer_256')(x)

x = layers.Dense(NUM_CLASSES, activation='softmax', name='dense_layer_120')(x)

model = keras.Model(inputs=inputs, outputs=x)
model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d_7   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense_layer_1024 (Dense)    (None, 1024)              1049600   
                                                           

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = "/content/gdrive/Shareddrives/MIRCV/Models/model_4layer1024512256120_Flatten_LR0001.h5" #il nome del modello è nel formato model_numerodese layer + sequenza dense layer_flatten oppure no_learning rate

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks=callbacks_list)

Epoch 1/25
258/258 [==============================] - ETA: 0s - loss: 1.8268 - accuracy: 0.5143
Epoch 1: val_loss improved from -inf to 0.84109, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_4layer1024512256120_Flatten_LR0001.h5
258/258 [==============================] - 83s 294ms/step - loss: 1.8268 - accuracy: 0.5143 - val_loss: 0.8411 - val_accuracy: 0.7225
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 1.0183 - accuracy: 0.6890
Epoch 2: val_loss did not improve from 0.84109
258/258 [==============================] - 75s 285ms/step - loss: 1.0183 - accuracy: 0.6890 - val_loss: 0.7942 - val_accuracy: 0.7485
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 0.8992 - accuracy: 0.7207
Epoch 3: val_loss did not improve from 0.84109
258/258 [==============================] - 75s 287ms/step - loss: 0.8992 - accuracy: 0.7207 - val_loss: 0.7715 - val_accuracy: 0.7549
Epoch 4/25
258/258 [==============================] - ETA: 0s - l

In [ ]:
'''
Epoch 1/25
258/258 [==============================] - ETA: 0s - loss: 1.8268 - accuracy: 0.5143
Epoch 1: val_loss improved from -inf to 0.84109, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_4layer1024512256120_Flatten_LR0001.h5
258/258 [==============================] - 83s 294ms/step - loss: 1.8268 - accuracy: 0.5143 - val_loss: 0.8411 - val_accuracy: 0.7225
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 1.0183 - accuracy: 0.6890
Epoch 2: val_loss did not improve from 0.84109
258/258 [==============================] - 75s 285ms/step - loss: 1.0183 - accuracy: 0.6890 - val_loss: 0.7942 - val_accuracy: 0.7485
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 0.8992 - accuracy: 0.7207
Epoch 3: val_loss did not improve from 0.84109
258/258 [==============================] - 75s 287ms/step - loss: 0.8992 - accuracy: 0.7207 - val_loss: 0.7715 - val_accuracy: 0.7549
Epoch 4/25
258/258 [==============================] - ETA: 0s - loss: 0.8143 - accuracy: 0.7422
Epoch 4: val_loss did not improve from 0.84109
258/258 [==============================] - 75s 286ms/step - loss: 0.8143 - accuracy: 0.7422 - val_loss: 0.7603 - val_accuracy: 0.7602
Epoch 5/25
258/258 [==============================] - ETA: 0s - loss: 0.7637 - accuracy: 0.7656
Epoch 5: val_loss did not improve from 0.84109
258/258 [==============================] - 75s 287ms/step - loss: 0.7637 - accuracy: 0.7656 - val_loss: 0.7583 - val_accuracy: 0.7660
Epoch 6/25
258/258 [==============================] - ETA: 0s - loss: 0.7200 - accuracy: 0.7747
Epoch 6: val_loss did not improve from 0.84109
258/258 [==============================] - 75s 287ms/step - loss: 0.7200 - accuracy: 0.7747 - val_loss: 0.7320 - val_accuracy: 0.7777
Epoch 7/25
258/258 [==============================] - ETA: 0s - loss: 0.6731 - accuracy: 0.7845
Epoch 7: val_loss did not improve from 0.84109
258/258 [==============================] - 75s 285ms/step - loss: 0.6731 - accuracy: 0.7845 - val_loss: 0.7521 - val_accuracy: 0.7699
Epoch 8/25
258/258 [==============================] - ETA: 0s - loss: 0.6269 - accuracy: 0.7972
Epoch 8: val_loss did not improve from 0.84109
258/258 [==============================] - 80s 306ms/step - loss: 0.6269 - accuracy: 0.7972 - val_loss: 0.7722 - val_accuracy: 0.7631
'''

# **EXPERIMENT 11**

In [ ]:
############################################
#### FATTO, HO LEARNING RATE #####
############################################

inputs = keras.Input(shape=(224,224,3))
x = data_augmentation (inputs)
x = pretrained_model(x)

x = layers.GlobalAveragePooling2D() (x)

x = layers.Flatten(name='flatten')(x)

x = layers.Dense(1024, activation='relu', name='dense_layer_1024')(x)

x = layers.Dense(512, activation='relu', name='dense_layer_512')(x)

x = layers.Dense(256, activation='relu', name='dense_layer_256')(x)

x = layers.Dense(NUM_CLASSES, activation='softmax', name='dense_layer_120')(x)

model = keras.Model(inputs=inputs, outputs=x)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense_layer_1024 (Dense)    (None, 1024)              1049600   
                                                             

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = "/content/gdrive/Shareddrives/MIRCV/Models/model_4layer1024512256120_Flatten_LR0001.h5" #il nome del modello è nel formato model_numerodese layer + sequenza dense layer_flatten oppure no_learning rate

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks=callbacks_list)

Epoch 1/25
258/258 [==============================] - ETA: 0s - loss: 2.6876 - accuracy: 0.3197
Epoch 1: val_loss improved from -inf to 1.34041, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_4layer1024512256120_Flatten_LR0001.h5
258/258 [==============================] - 1834s 7s/step - loss: 2.6876 - accuracy: 0.3197 - val_loss: 1.3404 - val_accuracy: 0.5734
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 1.3676 - accuracy: 0.5850
Epoch 2: val_loss did not improve from 1.34041
258/258 [==============================] - 80s 302ms/step - loss: 1.3676 - accuracy: 0.5850 - val_loss: 1.1250 - val_accuracy: 0.6453
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 1.2096 - accuracy: 0.6321
Epoch 3: val_loss did not improve from 1.34041
258/258 [==============================] - 80s 303ms/step - loss: 1.2096 - accuracy: 0.6321 - val_loss: 0.9503 - val_accuracy: 0.7029
Epoch 4/25
258/258 [==============================] - ETA: 0s - lo

# **EXPERIMENT 12**

In [ ]:
############################################
#### FATTO CAMBIATO BATCH SIZE #####
############################################

inputs = keras.Input(shape=(224,224,3))
x = data_augmentation (inputs)
x = pretrained_model(x)

x = layers.GlobalAveragePooling2D() (x)

x = layers.Flatten(name='flatten')(x)

x = layers.Dense(1024, activation='relu', name='dense_layer_1024')(x)

x = layers.Dense(512, activation='relu', name='dense_layer_512')(x)

x = layers.Dense(256, activation='relu', name='dense_layer_256')(x)

x = layers.Dense(NUM_CLASSES, activation='softmax', name='dense_layer_120')(x)

model = keras.Model(inputs=inputs, outputs=x)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d_2   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense_layer_1024 (Dense)    (None, 1024)              1049600   
                                                           

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = "/content/gdrive/Shareddrives/MIRCV/Models/model_4layer1024512256120_Flatten_LR0001.h5" #il nome del modello è nel formato model_numerodese layer + sequenza dense layer_flatten oppure no_learning rate

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks=callbacks_list)

Epoch 1/25
258/258 [==============================] - ETA: 0s - loss: 2.9332 - accuracy: 0.2803
Epoch 1: val_loss improved from -inf to 1.35243, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_4layer1024512256120_Flatten_LR0001.h5
258/258 [==============================] - 86s 307ms/step - loss: 2.9332 - accuracy: 0.2803 - val_loss: 1.3524 - val_accuracy: 0.5734
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 1.4132 - accuracy: 0.5698
Epoch 2: val_loss did not improve from 1.35243
258/258 [==============================] - 78s 296ms/step - loss: 1.4132 - accuracy: 0.5698 - val_loss: 1.1348 - val_accuracy: 0.6363
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 1.2519 - accuracy: 0.6187
Epoch 3: val_loss did not improve from 1.35243
258/258 [==============================] - 78s 297ms/step - loss: 1.2519 - accuracy: 0.6187 - val_loss: 1.0150 - val_accuracy: 0.6778
Epoch 4/25
258/258 [==============================] - ETA: 0s - l

# **EXPERIMENT** **13**

In [ ]:
############################################
#### FATTO #####
############################################

inputs = keras.Input(shape=(224,224,3))
x = data_augmentation (inputs)
x = pretrained_model(x)

x = layers.GlobalAveragePooling2D() (x)

x = layers.Flatten(name='flatten')(x)

x = layers.Dense(1024, activation='relu', name='dense_layer_1024')(x)

x = layers.Dense(NUM_CLASSES, activation='softmax', name='dense_layer_120')(x)

model = keras.Model(inputs=inputs, outputs=x)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense_layer_1024 (Dense)    (None, 1024)              1049600   
                                                             

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = "/content/gdrive/Shareddrives/MIRCV/Models/model_5layer1024120_Flatten_LR0001.h5" #il nome del modello è nel formato model_numerodese layer + sequenza dense layer_flatten oppure no_learning rate

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks=callbacks_list)

Epoch 1/25
515/515 [==============================] - ETA: 0s - loss: 1.4738 - accuracy: 0.6018
Epoch 1: val_loss improved from -inf to 0.76885, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_5layer1024120_Flatten_LR0001.h5
515/515 [==============================] - 2771s 5s/step - loss: 1.4738 - accuracy: 0.6018 - val_loss: 0.7688 - val_accuracy: 0.7636
Epoch 2/25
515/515 [==============================] - ETA: 0s - loss: 0.8780 - accuracy: 0.7297
Epoch 2: val_loss did not improve from 0.76885
515/515 [==============================] - 81s 155ms/step - loss: 0.8780 - accuracy: 0.7297 - val_loss: 0.7125 - val_accuracy: 0.7772
Epoch 3/25
515/515 [==============================] - ETA: 0s - loss: 0.7623 - accuracy: 0.7608
Epoch 3: val_loss did not improve from 0.76885
515/515 [==============================] - 86s 167ms/step - loss: 0.7623 - accuracy: 0.7608 - val_loss: 0.7605 - val_accuracy: 0.7736
Epoch 4/25
515/515 [==============================] - ETA: 0s - loss: 0.

# **EXPERIMENT** **14**

In [ ]:
############################################
#### FATTO #####
############################################

inputs = keras.Input(shape=(224,224,3))
x = data_augmentation (inputs)
x = pretrained_model(x)

x = layers.GlobalAveragePooling2D() (x)

x = layers.Flatten(name='flatten')(x)

x = layers.Dense(1024, activation='relu', name='dense_layer_1024')(x)

x = layers.Dense(512, activation='relu', name='dense_layer_512')(x)

x = layers.Dense(NUM_CLASSES, activation='softmax', name='dense_layer_120')(x)

model = keras.Model(inputs=inputs, outputs=x)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d_1   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense_layer_1024 (Dense)    (None, 1024)              1049600   
                                                           

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = "/content/gdrive/Shareddrives/MIRCV/Models/model_6layer1024512120_Flatten_L0005.h5" #il nome del modello è nel formato model_numerodese layer + sequenza dense layer_flatten oppure no_learning rate

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks=callbacks_list)

Epoch 1/25
515/515 [==============================] - ETA: 0s - loss: 2.0458 - accuracy: 0.4535
Epoch 1: val_loss improved from -inf to 1.09911, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_6layer1024512120_Flatten_L0005.h5
515/515 [==============================] - 92s 167ms/step - loss: 2.0458 - accuracy: 0.4535 - val_loss: 1.0991 - val_accuracy: 0.6555
Epoch 2/25
515/515 [==============================] - ETA: 0s - loss: 1.3218 - accuracy: 0.6087
Epoch 2: val_loss did not improve from 1.09911
515/515 [==============================] - 96s 185ms/step - loss: 1.3218 - accuracy: 0.6087 - val_loss: 1.0777 - val_accuracy: 0.6761
Epoch 3/25
515/515 [==============================] - ETA: 0s - loss: 1.2355 - accuracy: 0.6333
Epoch 3: val_loss did not improve from 1.09911
515/515 [==============================] - 80s 155ms/step - loss: 1.2355 - accuracy: 0.6333 - val_loss: 1.0424 - val_accuracy: 0.6863
Epoch 4/25
515/515 [==============================] - ETA: 0s - loss:

# **EXPERIMENT 15**

In [ ]:
############################################
#### FATTO #####
############################################

inputs = keras.Input(shape=(224,224,3))
x = data_augmentation (inputs)
x = pretrained_model(x)

x = layers.GlobalAveragePooling2D() (x)

x = layers.Flatten(name='flatten')(x)

x = layers.Dense(512, activation='relu', name='dense_layer_512')(x)

x = layers.Dense(NUM_CLASSES, activation='softmax', name='dense_layer_120')(x)

model = keras.Model(inputs=inputs, outputs=x)
model.summary()

KeyboardInterrupt: ignored

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d_4   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense_layer_512 (Dense)     (None, 512)               524800    
                                                           

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = "/content/gdrive/Shareddrives/MIRCV/Models/model_7layer512120_Flatten_L0005.h5" #il nome del modello è nel formato model_numerodese layer + sequenza dense layer_flatten oppure no_learning rate

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks=callbacks_list)

Epoch 1/25
515/515 [==============================] - ETA: 0s - loss: 1.6864 - accuracy: 0.5433
Epoch 1: val_loss improved from -inf to 1.00791, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_7layer512120_Flatten_L0005.h5
515/515 [==============================] - 92s 167ms/step - loss: 1.6864 - accuracy: 0.5433 - val_loss: 1.0079 - val_accuracy: 0.6922
Epoch 2/25
515/515 [==============================] - ETA: 0s - loss: 1.1583 - accuracy: 0.6538
Epoch 2: val_loss improved from 1.00791 to 1.03519, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_7layer512120_Flatten_L0005.h5
515/515 [==============================] - 84s 161ms/step - loss: 1.1583 - accuracy: 0.6538 - val_loss: 1.0352 - val_accuracy: 0.6975
Epoch 3/25
515/515 [==============================] - ETA: 0s - loss: 1.0890 - accuracy: 0.6760
Epoch 3: val_loss did not improve from 1.03519
515/515 [==============================] - 80s 154ms/step - loss: 1.0890 - accuracy: 0.6760 - val_loss: 0.98

# **EXPERIMENT 16**

In [ ]:
############################################
#### FATTO #####
############################################

inputs = keras.Input(shape=(224,224,3))
x = data_augmentation (inputs)
x = pretrained_model(x)

x = layers.GlobalAveragePooling2D() (x)

x = layers.Flatten(name='flatten')(x)

x = layers.Dense(256, activation='relu', name='dense_layer_256')(x)

x = layers.Dense(NUM_CLASSES, activation='softmax', name='dense_layer_120')(x)

model = keras.Model(inputs=inputs, outputs=x)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d_3   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense_layer_256 (Dense)     (None, 256)               262400    
                                                           

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = "/content/gdrive/Shareddrives/MIRCV/Models/model_8layer256120_Flatten_L0005.h5" #il nome del modello è nel formato model_numerodese layer + sequenza dense layer_flatten oppure no_learning rate

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks=callbacks_list)

Epoch 1/25
515/515 [==============================] - ETA: 0s - loss: 1.7436 - accuracy: 0.5247
Epoch 1: val_loss improved from -inf to 0.96628, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_8layer256120_Flatten_L0005.h5
515/515 [==============================] - 94s 171ms/step - loss: 1.7436 - accuracy: 0.5247 - val_loss: 0.9663 - val_accuracy: 0.7031
Epoch 2/25
515/515 [==============================] - ETA: 0s - loss: 1.1754 - accuracy: 0.6474
Epoch 2: val_loss did not improve from 0.96628
515/515 [==============================] - 87s 168ms/step - loss: 1.1754 - accuracy: 0.6474 - val_loss: 0.9384 - val_accuracy: 0.7126
Epoch 3/25
515/515 [==============================] - ETA: 0s - loss: 1.1147 - accuracy: 0.6689
Epoch 3: val_loss did not improve from 0.96628
515/515 [==============================] - 85s 164ms/step - loss: 1.1147 - accuracy: 0.6689 - val_loss: 0.9355 - val_accuracy: 0.7160
Epoch 4/25
515/515 [==============================] - ETA: 0s - loss: 1.0

# **EXPERIMENT 17**

In [ ]:
############################################
#### DA FARE #####
############################################

inputs = keras.Input(shape=(224,224,3))
x = data_augmentation (inputs)
x = pretrained_model(x)

x = layers.GlobalAveragePooling2D() (x)

x = layers.Flatten(name='flatten')(x)

x = layers.Dense(512, activation='relu', name='dense_layer_512')(x)

x = layers.Dense(256, activation='relu', name='dense_layer_256')(x)

x = layers.Dense(NUM_CLASSES, activation='softmax', name='dense_layer_120')(x)

model = keras.Model(inputs=inputs, outputs=x)
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d_5   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense_layer_512 (Dense)     (None, 512)               524800    
                                                           

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = "/content/gdrive/Shareddrives/MIRCV/Models/model_9layer512256120_Flatten_L0005.h5" #il nome del modello è nel formato model_numerodese layer + sequenza dense layer_flatten oppure no_learning rate

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks=callbacks_list)

Epoch 1/25
515/515 [==============================] - ETA: 0s - loss: 2.0377 - accuracy: 0.4490
Epoch 1: val_loss improved from -inf to 1.10618, saving model to /content/gdrive/Shareddrives/MIRCV/Models/model_9layer512256120_Flatten_L0005.h5
515/515 [==============================] - 89s 160ms/step - loss: 2.0377 - accuracy: 0.4490 - val_loss: 1.1062 - val_accuracy: 0.6545
Epoch 2/25
515/515 [==============================] - ETA: 0s - loss: 1.3115 - accuracy: 0.6078
Epoch 2: val_loss did not improve from 1.10618
515/515 [==============================] - 81s 157ms/step - loss: 1.3115 - accuracy: 0.6078 - val_loss: 1.0626 - val_accuracy: 0.6696
Epoch 3/25
515/515 [==============================] - ETA: 0s - loss: 1.2210 - accuracy: 0.6335
Epoch 3: val_loss did not improve from 1.10618
515/515 [==============================] - 82s 157ms/step - loss: 1.2210 - accuracy: 0.6335 - val_loss: 1.0447 - val_accuracy: 0.6844
Epoch 4/25
515/515 [==============================] - ETA: 0s - loss: 

# **EXPERIMENT 3 DIFFERENT MODEL**

In [ ]:
x = data_augmentation
x = pretrained_model.output

x = layers.GlobalAveragePooling2D() (x)

x = layers.Flatten(name='flatten')(x)

x = layers.Dense(512, activation='relu', name='dense_layer_512')(x)

x = layers.Dense(256, activation='relu', name='dense_layer_256')(x)

x = layers.Dense(NUM_CLASSES, activation='softmax', name='dense_layer_120')(x)

model = tf.keras.models.Model(inputs=pretrained_model.input, outputs=x, name='Dogs_Image_Model_512256120_LR0001')

model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = "/content/gdrive/Shareddrives/MIRCV/Models/Model_different_512256120_LR0001" #il nome del modello è nel formato model_numerodese layer + sequenza dense layer_flatten oppure no_learning rate

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks=callbacks_list)

Epoch 1/25
515/515 [==============================] - ETA: 0s - loss: 1.3284 - accuracy: 0.6364
Epoch 1: val_loss improved from -inf to 0.81818, saving model to /content/gdrive/Shareddrives/MIRCV/Models/Model_different_512256120_LR0001
515/515 [==============================] - 88s 160ms/step - loss: 1.3284 - accuracy: 0.6364 - val_loss: 0.8182 - val_accuracy: 0.7398
Epoch 2/25
515/515 [==============================] - ETA: 0s - loss: 0.6369 - accuracy: 0.7914
Epoch 2: val_loss did not improve from 0.81818
515/515 [==============================] - 81s 155ms/step - loss: 0.6369 - accuracy: 0.7914 - val_loss: 0.6884 - val_accuracy: 0.7794
Epoch 3/25
515/515 [==============================] - ETA: 0s - loss: 0.5076 - accuracy: 0.8313
Epoch 3: val_loss did not improve from 0.81818
515/515 [==============================] - 79s 153ms/step - loss: 0.5076 - accuracy: 0.8313 - val_loss: 0.7070 - val_accuracy: 0.7804
Epoch 4/25
515/515 [==============================] - ETA: 0s - loss: 0.4065

In [ ]:
model.save('/content/gdrive/Shareddrives/MIRCV/Models/Prova_Save_Model_different_Flatten_512256120_LR0001')

INFO:tensorflow:Assets written to: /content/gdrive/Shareddrives/MIRCV/Models/Prova_Save_Model_different_Flatten_512256120_LR0001/assets


# **EXPERIMENT 5 DIFFERENT MODEL**

In [ ]:
x = data_augmentation
x = pretrained_model.output

x = layers.GlobalAveragePooling2D() (x)

x = layers.Dense(512, activation='relu', name='dense_layer_512')(x)

x = layers.Dense(NUM_CLASSES, activation='softmax', name='dense_layer_120')(x)

model = tf.keras.models.Model(inputs=pretrained_model.input, outputs=x, name='Dogs_Image_Model_512256120_LR0001')

model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = "/content/gdrive/Shareddrives/MIRCV/Models/Model_different_512120_LR0001" #il nome del modello è nel formato model_numerodese layer + sequenza dense layer_flatten oppure no_learning rate

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks=callbacks_list)

Epoch 1/25
515/515 [==============================] - ETA: 0s - loss: 1.2092 - accuracy: 0.6715
Epoch 1: val_loss improved from -inf to 0.72325, saving model to /content/gdrive/Shareddrives/MIRCV/Models/Model_different_512120_LR0001
515/515 [==============================] - 2599s 5s/step - loss: 1.2092 - accuracy: 0.6715 - val_loss: 0.7233 - val_accuracy: 0.7750
Epoch 2/25
515/515 [==============================] - ETA: 0s - loss: 0.5449 - accuracy: 0.8276
Epoch 2: val_loss did not improve from 0.72325
515/515 [==============================] - 78s 150ms/step - loss: 0.5449 - accuracy: 0.8276 - val_loss: 0.7044 - val_accuracy: 0.7799
Epoch 3/25
515/515 [==============================] - ETA: 0s - loss: 0.4273 - accuracy: 0.8567
Epoch 3: val_loss improved from 0.72325 to 0.72849, saving model to /content/gdrive/Shareddrives/MIRCV/Models/Model_different_512120_LR0001
515/515 [==============================] - 95s 182ms/step - loss: 0.4273 - accuracy: 0.8567 - val_loss: 0.7285 - val_accu

In [ ]:
model.save('/content/gdrive/Shareddrives/MIRCV/Models/Prova_Save_Model_different_512120_LR0001')

INFO:tensorflow:Assets written to: /content/gdrive/Shareddrives/MIRCV/Models/Prova_Save_Model_different_512120_LR0001/assets


# **EXPERIMENT 6 DIFFERENT MODEL**

In [ ]:
x = data_augmentation
x = pretrained_model.output

x = layers.GlobalAveragePooling2D() (x)

x = layers.Flatten(name='flatten')(x)

x = layers.Dense(512, activation='relu', name='dense_layer_512')(x)

x = layers.Dense(NUM_CLASSES, activation='softmax', name='dense_layer_120')(x)

model = tf.keras.models.Model(inputs=pretrained_model.input, outputs=x, name='Dogs_Image_Model_512256120_LR0001')

model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = "/content/gdrive/Shareddrives/MIRCV/Models/Model_different_flatten_512120_LR0001" #il nome del modello è nel formato model_numerodese layer + sequenza dense layer_flatten oppure no_learning rate

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks=callbacks_list)

Epoch 1/25
515/515 [==============================] - ETA: 0s - loss: 1.1871 - accuracy: 0.6751
Epoch 1: val_loss improved from -inf to 0.75847, saving model to /content/gdrive/Shareddrives/MIRCV/Models/Model_different_flatten_512120_LR0001
515/515 [==============================] - 92s 167ms/step - loss: 1.1871 - accuracy: 0.6751 - val_loss: 0.7585 - val_accuracy: 0.7604
Epoch 2/25
515/515 [==============================] - ETA: 0s - loss: 0.5547 - accuracy: 0.8208
Epoch 2: val_loss did not improve from 0.75847
515/515 [==============================] - 84s 163ms/step - loss: 0.5547 - accuracy: 0.8208 - val_loss: 0.7001 - val_accuracy: 0.7930
Epoch 3/25
515/515 [==============================] - ETA: 0s - loss: 0.4268 - accuracy: 0.8585
Epoch 3: val_loss did not improve from 0.75847
515/515 [==============================] - 79s 153ms/step - loss: 0.4268 - accuracy: 0.8585 - val_loss: 0.7012 - val_accuracy: 0.7930
Epoch 4/25
515/515 [==============================] - ETA: 0s - loss: 0

In [ ]:
model.save('/content/gdrive/Shareddrives/MIRCV/Models/Prova_Save_Model_different_Flatten_512120_LR0001')

INFO:tensorflow:Assets written to: /content/gdrive/Shareddrives/MIRCV/Models/Prova_Save_Model_different_Flatten_512120_LR0001/assets


# **EXPERIMENT 6 FINE TUNING**

In [ ]:
model= keras.models.load_model('/content/gdrive/Shareddrives/MIRCV/Models/Prova_Save_Model_different_Flatten_512120_LR0001')

In [ ]:
model.load_weights("/content/gdrive/Shareddrives/MIRCV/Models/Prova_Save_Model_different_Flatten_512120_LR0001")
model.summary()

Model: "Dogs_Image_Model_512256120_LR0001"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                  

In [ ]:
trainable = False
for layer in model.layers:
    if layer.name == 'conv5_block15_0_bn':
        trainable = True
    is_bn = layer.name.endswith('bn')  # keep BatchNorm freezed, good practice for finetuned models
    if not is_bn:
      layer.trainable = trainable

model.summary()

Model: "Dogs_Image_Model_512256120_LR0001"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                  

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = '/content/gdrive/Shareddrives/MIRCV/Models/Fine_Tuned_different_model_Flatten_512120_LR0001'

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history_fn = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks = callbacks_list
                    )

Epoch 1/25
515/515 [==============================] - ETA: 0s - loss: 0.3342 - accuracy: 0.8867

In [ ]:
model.save('/content/gdrive/Shareddrives/MIRCV/Models/Fine_Tuning_Model6_Flatten_512120_LR0001')

# **EXPERIMENT 5 FINE TUNING**

In [ ]:
model= keras.models.load_model('/content/gdrive/Shareddrives/MIRCV/Models/Prova_Save_Model_different_512120_LR0001')

In [ ]:
model.load_weights('/content/gdrive/Shareddrives/MIRCV/Models/Model_different_512120_LR0001')
model.summary()

Model: "Dogs_Image_Model_512256120_LR0001"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                  

In [ ]:
trainable = False
for layer in model.layers:
    if layer.name == 'conv5_block15_0_bn':
        trainable = True
    is_bn = layer.name.endswith('bn')  # keep BatchNorm freezed, good practice for finetuned models
    if not is_bn:
      layer.trainable = trainable

model.summary()

Model: "Dogs_Image_Model_512256120_LR0001"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                  

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = "/content/gdrive/Shareddrives/MIRCV/Models/Model_different_512120_LR0001" #il nome del modello è nel formato model_numerodese layer + sequenza dense layer_flatten oppure no_learning rate

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks=callbacks_list)

Epoch 1/25
515/515 [==============================] - ETA: 0s - loss: 0.4000 - accuracy: 0.8661
Epoch 1: val_loss improved from -inf to 0.70255, saving model to /content/gdrive/Shareddrives/MIRCV/Models/Model_different_512120_LR0001
515/515 [==============================] - 3583s 7s/step - loss: 0.4000 - accuracy: 0.8661 - val_loss: 0.7026 - val_accuracy: 0.7879
Epoch 2/25
515/515 [==============================] - ETA: 0s - loss: 0.2909 - accuracy: 0.9006
Epoch 2: val_loss improved from 0.70255 to 0.72664, saving model to /content/gdrive/Shareddrives/MIRCV/Models/Model_different_512120_LR0001
515/515 [==============================] - 82s 158ms/step - loss: 0.2909 - accuracy: 0.9006 - val_loss: 0.7266 - val_accuracy: 0.7918
Epoch 3/25
515/515 [==============================] - ETA: 0s - loss: 0.2280 - accuracy: 0.9228
Epoch 3: val_loss improved from 0.72664 to 0.79753, saving model to /content/gdrive/Shareddrives/MIRCV/Models/Model_different_512120_LR0001
515/515 [===================

In [ ]:
model.save('/content/gdrive/Shareddrives/MIRCV/Models/Fine_Tuning_Model5_512120_LR0001')

INFO:tensorflow:Assets written to: /content/gdrive/Shareddrives/MIRCV/Models/Fine_Tuning_Model5_512120_LR0001/assets


# **EXPERIMENT 3 FINE TUNING**

In [ ]:
model = keras.models.load_model('/content/gdrive/Shareddrives/MIRCV/Models/Prova_Save_Model_different_Flatten_512256120_LR0001')

In [ ]:
model.load_weights("/content/gdrive/Shareddrives/MIRCV/Models/Model_different_512256120_LR0001")
model.summary()

Model: "Dogs_Image_Model_512256120_LR0001"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                  

In [ ]:
trainable = False
for layer in model.layers:
    if layer.name == 'conv5_block15_0_bn':
        trainable = True
    is_bn = layer.name.endswith('bn')  # keep BatchNorm freezed, good practice for finetuned models
    if not is_bn:
      layer.trainable = trainable

model.summary()

Model: "Dogs_Image_Model_512256120_LR0001"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                  

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])   # si prova con categorical e poi con sparse_categorical

path = "/content/gdrive/Shareddrives/MIRCV/Models/Model_different_512256120_LR0001" #il nome del modello è nel formato model_numerodese layer + sequenza dense layer_flatten oppure no_learning rate

callbacks_list = [
                  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
                  keras.callbacks.ModelCheckpoint(
                      filepath = path,
                      monitor="val_loss",
                      verbose=1,
                      save_best_only=True,
                      mode='max',
                      save_weights_only=True),
                  ]

history = model.fit(train_ds,
                    epochs=25,
                    #steps_per_epoch = len(train_ds),
                    validation_data=val_ds,
                    #validation_steps = len(val_ds),
                    verbose = 1,
                    callbacks=callbacks_list)

Epoch 1/25
515/515 [==============================] - ETA: 0s - loss: 0.6998 - accuracy: 0.7749
Epoch 1: val_loss improved from -inf to 0.74773, saving model to /content/gdrive/Shareddrives/MIRCV/Models/Model_different_512256120_LR0001
515/515 [==============================] - 90s 161ms/step - loss: 0.6998 - accuracy: 0.7749 - val_loss: 0.7477 - val_accuracy: 0.7668
Epoch 2/25
515/515 [==============================] - ETA: 0s - loss: 0.5238 - accuracy: 0.8248
Epoch 2: val_loss improved from 0.74773 to 0.76084, saving model to /content/gdrive/Shareddrives/MIRCV/Models/Model_different_512256120_LR0001
515/515 [==============================] - 82s 158ms/step - loss: 0.5238 - accuracy: 0.8248 - val_loss: 0.7608 - val_accuracy: 0.7575
Epoch 3/25
515/515 [==============================] - ETA: 0s - loss: 0.4159 - accuracy: 0.8618
Epoch 3: val_loss improved from 0.76084 to 0.76401, saving model to /content/gdrive/Shareddrives/MIRCV/Models/Model_different_512256120_LR0001
515/515 [=========

In [ ]:
model.save('/content/gdrive/Shareddrives/MIRCV/Models/Fine_Tuning_Model3_512120_LR0001')

INFO:tensorflow:Assets written to: /content/gdrive/Shareddrives/MIRCV/Models/Fine_Tuning_Model3_512120_LR0001/assets
